## 코딩절차
1.cx_Oracle 모듈 import(파이썬에서 오라클dbms와 연동하는 다양한 함수를 제공하는 모듈)

2. cx_Oracle모듈의 connect()함수를 이용해 db연결

3. connection 객체의 cursor 메소드를 이용해 cursor 객체 조회
    - 커서객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체/
    
4. cursor의 execute()메소드를 이용해 sql문 전송

5. select의 경우 결과 조회 : cursor의 fetch 메소드사용

6. insert,delete,upgrade의 경우 sql문 실행 후 커넥션의 커밋 메소드를 통해 transaction

7. connection, cursor 연결닫기


## connection : db연결


In [2]:
import cx_Oracle
username = "c##scott_join"
password = "tiger"
url = "localhost:1521/XE" #HOST:PORT/SID
conn = cx_Oracle.connect(username,password,url)
print(type(conn))
conn.close()

<class 'cx_Oracle.Connection'>


In [3]:
conn2 = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
print(type(conn2))
conn2.close()

<class 'cx_Oracle.Connection'>


In [4]:
username = "c##scott_join" # 연결방법 3
password = "tiger"
host = "localhost"
port = 1521
sid = 'XE'

dsn = cx_Oracle.makedsn(host,port,sid) #dsn 생성
print(dsn)
conn3 = cx_Oracle.connect(username, password, dsn)
print(type(conn3))
conn3.close()

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))
<class 'cx_Oracle.Connection'>


In [6]:
#with문 : 빠져나올 때 자동 close
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn4:
    pass

## cursor 객체
- sql문을 전송하고 select 결과를 조회하는 메소드를 제공
- connection 객체의 cursor()메소드로 받아온다.

### CURSOR의 주요 메소드
- SQL 실행 메소드 : EXECUTE(SQL), EXECUTEMANY(SQL) - 배치처리
- SELECT 결과조회 메소드 : fetchall(), fetchone(), fetchmany(n)

In [8]:
import cx_Oracle
from pprint import pprint

In [14]:
select_sql = "SELECT emp_id,emp_name,job_id FROM emp"
insert_sql = "INSERT INTO emp Values (1501,'홍길동','FI_ACCOUNT',101, to_date('2021/02/15','yyyy/mm/dd'), 30000, 0.2, 100,'M')"

conn = None
cursor = None

#insert 후 SELECT

try:
    
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') 
    print("연결완료")
    cursor = conn.cursor()
#sql 실행
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
#select 결과 조회
    result = cursor.fetchall()
    print(type(result),type(result[0]))
    pprint(result)

except exception as e:
     print(e)
    
finally:
    if conn != None:
        conn.close()
    if cursor !=None:
        conn.close()

연결완료
<class 'list'> <class 'tuple'>
[(1000, '홍길동', 'FI_ACCOUNT'),
 (1100, '이순신', None),
 (1200, '박영희', 'FI_ACCOUNT'),
 (1300, '박영희', 'FI_ACCOUNT'),
 (1500, '홍길동', 'FI_ACCOUNT'),
 (1501, '홍길동', 'FI_ACCOUNT'),
 (100, 'Steven', None),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (116, 'Shelli', 'PU_CLERK'),
 (117, 'Sigal', 'PU_CLERK'),
 (118, 'Guy', 'PU_CLERK'),
 (119, 'Karen', 'PU_CLERK'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (125, 'Julia', 'ST_CLERK'),
 (126, 'Irene', 'ST_CLERK'),
 (127, 'James', 'ST_

InterfaceError: not connected

## placeholder(자리표시자)를 이용해 sql 실행
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식
    - 리스트나 튜플: placeholder 순서


In [16]:
select_sql = "SELECT * FROM emp WHERE salary BETWEEN :1 AND :2" #:1과 :2는 나중에 알려줄께(1부터 시작 0번인덱스 값이 1번으로 들어감)
insert_sql = "INSERT INTO emp VALUES\
(:id, :name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id, :gender)"

conn = None
cursor = None


try:
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE')
    cursor = conn.cursor()
    
    param_dict ={
        'id':2001,
        'name':'이순신',
        'job_id':'FI_ACCOUNT',
        'mgr_id':101,
        'hire_date':'20210213',
        'salary':19990,
        'comm_pct':None,
        'dept_id':110,
        'gender':'M'
        
    }
    cursor.execute(insert_sql, param_dict)
    cursor.execute(select_sql, [15000,20000])
    result1 = cursor.fetchall()
    print("15000~20000사이조회")
    pprint(result1)
except exception as e:
    print(e)
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

15000~20000사이조회
[(2001,
  '이순신',
  'FI_ACCOUNT',
  101,
  datetime.datetime(2021, 2, 13, 0, 0),
  19990.0,
  None,
  110,
  'M'),
 (101,
  'Neena',
  'AD_VP',
  100,
  datetime.datetime(2005, 9, 21, 0, 0),
  17000.0,
  None,
  90,
  None),
 (102,
  'Lex',
  'AD_VP',
  100,
  datetime.datetime(2001, 1, 13, 0, 0),
  17000.0,
  None,
  90,
  None),
 (104,
  'Bruce',
  'IT_PROG',
  103,
  datetime.datetime(2007, 5, 21, 0, 0),
  18000.0,
  None,
  60,
  None)]


## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.
- 한행 조회시 사용, 반환타입 : tuple

In [20]:
#emp_id(pk)로 직원정보 조회하는 함수
def select_emp_by_id(emp_id):
    sql = "SELECT * from emp WHERE emp_id = :1"
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            
            #sql 실행
            cursor.execute(sql, [emp_id])
            result = cursor.fetchone()
            return result

In [21]:
r = select_emp_by_id(100)

if r!=None:
    print(r, "id:", r[0], "이름:",r[1])
else:
    ("조회결과 없음")

(100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90, None) id: 100 이름: Steven


In [22]:
r = select_emp_by_id(101)

if r:
    print(r, "id:", r[0], "이름:",r[1])
    
else:
    ("조회결과 없음")

(101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 90, None) id: 101 이름: Neena


In [28]:
sql = "SELECT * FROM emp ORDER BY emp_id"

with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(30)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
     

10
[(100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90, None), (101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 90, None), (102, 'Lex', 'AD_VP', 100, datetime.datetime(2001, 1, 13, 0, 0), 17000.0, None, 90, None), (103, 'Alexander', 'IT_PROG', 102, datetime.datetime(2006, 1, 3, 0, 0), 27000.0, None, 60, None), (104, 'Bruce', 'IT_PROG', 103, datetime.datetime(2007, 5, 21, 0, 0), 18000.0, None, 60, None), (105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 14400.0, None, 60, None), (106, 'Valli', 'IT_PROG', 103, datetime.datetime(2006, 2, 5, 0, 0), 14400.0, None, 60, None), (107, 'Diana', 'IT_PROG', 103, datetime.datetime(2007, 2, 7, 0, 0), 12600.0, None, 60, None), (108, 'Nancy', 'FI_MGR', 101, datetime.datetime(2002, 8, 17, 0, 0), 12008.0, None, 100, None), (109, 'Daniel', 'FI_ACCOUNT', 108, datetime.datetime(2002, 8, 16, 0, 0), 9000.0, None, 100, None)]
10
[(100, 'Steven', None, 100, datetime.datetime(200

In [32]:
sql = "SELECT * FROM emp WHERE comm_pct IS NOT NULL ORDER BY emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        for idx, row in enumerate(cursor, start = 1): #   한번 반복시마다 cursor.fetchone 실행
            print(f"{idx},{row}")

1,(100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90, None)
2,(145, 'John', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 14000.0, 0.4, 80, None)
3,(146, 'Karen', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 13500.0, 0.3, 80, None)
4,(147, 'Alberto', 'SA_MAN', 100, datetime.datetime(2005, 3, 10, 0, 0), 12000.0, 0.3, 80, None)
5,(148, 'Gerald', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 11000.0, 0.3, 80, None)
6,(149, 'Eleni', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 10500.0, 0.2, 80, None)
7,(150, 'Peter', 'SA_REP', 145, datetime.datetime(2007, 10, 15, 0, 0), 10000.0, 0.3, 80, None)
8,(151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 9500.0, 0.25, 80, None)
9,(152, 'Peter', 'SA_REP', 145, datetime.datetime(2005, 8, 20, 0, 0), 9000.0, 0.25, 80, None)
10,(153, 'Christopher', 'SA_REP', 145, datetime.datetime(2006, 3, 30, 0, 0), 8000.0, 0.2, 80, None)
11,(154, 'Nanette', 'SA_REP', 145, datetime.datetime